In [ ]:
#==========
# Load team-based statistics
#==========

'''
Pull in the oline, quarterback, and overall team offense statistics, and join them 
to the player data. This will provide team-based context for the players, as well as
allow for grouped statistics generation.
'''

# pull in team based stats for oline and quarterback play
oline = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Data/CSV/oline_rankings.csv', index_col=0)
qb = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Data/CSV/qb_stats_for_wr.csv', index_col=0)
team_off = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Data/CSV/team_offensive_efficiency.csv', index_col=0)
team_stats = pd.read_csv('/Users/Mark/Desktop/Jupyter Projects/Fantasy Football/Data/CSV/traditional_team_stats.csv', index_col=0)

# give qb attempts a unique name before merging
qb = qb.rename(columns={'att': 'qb_att'})

# merge all df with the original
team_files = [oline, qb, team_off, team_stats]
for file in team_files:
    df_merged = pd.merge(df_merged, file, how='left', left_on=['team', 'year'], right_on=['team', 'year'])
    
#==========
# Creating team based grouped statistics
#==========

'''
Create grouped statistics based on the team and teammates. For example,
create total touches by team feature, as well as how the average, min, and max
teammate adps compare to the current player.
'''

# groupby team and year to get total wr yard and targets for each team
team_tgts = df_merged.groupby(['team', 'year'], group_keys=False)['tgt'].agg(np.sum).reset_index().rename(columns={'tgt': 'tgt_on_team'})
df_merged = pd.merge(df_merged, team_tgts, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
df_merged['available_tgt'] = 1-(df_merged['tgt_on_team'] / df_merged['tm_pass_att'])
df_merged['available_tgt_2'] = 1-((df_merged['tgt_on_team'] - df_merged['tgt']) / df_merged['tm_pass_att'])

team_yds = df_merged.groupby(['team', 'year'], group_keys=False)['yds'].agg(np.sum).reset_index().rename(columns={'yds': 'yds_on_team'})
df_merged = pd.merge(df_merged, team_yds, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
df_merged['available_yds'] = 1-(df_merged['yds_on_team'] / df_merged['tm_pass_yds'])
df_merged['available_yds_2'] = 1-((df_merged['yds_on_team']-df_merged['yds']) / df_merged['tm_pass_yds'])

# create marketshare statistics
df_merged['ms_rec_yd'] = df_merged['yds'] / df_merged['tm_pass_yds']
df_merged['ms_tgts'] = df_merged['tgt'] / df_merged['tm_pass_att']
df_merged['ms_yds_per_tgts'] = df_merged['ms_rec_yd'] / df_merged['ms_tgts']

df_merged['avail_tgt_x_newteam'] = df_merged['available_tgt'] * df_merged['new_team']
df_merged['avail_yds_x_newteam'] = df_merged['available_yds'] * df_merged['new_team']

# create teammate ADP metrics to see if top ranked player
min_teammate = df_merged.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.min).reset_index().rename(columns={'avg_pick': 'min_teammate'})
max_teammate = df_merged.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.max).reset_index().rename(columns={'avg_pick': 'max_teammate'})
avg_teammate = df_merged.groupby(['team', 'year'], group_keys=False)['avg_pick'].agg(np.mean).reset_index().rename(columns={'avg_pick': 'avg_teammate'})

names = ['min_teammate', 'max_teammate', 'avg_teammate']
for i, file in enumerate([min_teammate, max_teammate, avg_teammate]):
    
    df_merged = pd.merge(df_merged, file, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
    df_merged['teammate_diff'] = df_merged[names[i]] - df_merged['min_teammate'] 